# Pre-process EBSD patterns from an AlMn alloy

300 C #1

Håkon Wiik Ånes (hakon.w.anes@ntnu.no)

In [33]:
%matplotlib qt5

from datetime import date
import os
from time import time

import dask
from dask.diagnostics import ProgressBar
from diffpy.structure import Lattice, Structure
from diffsims.crystallography import ReciprocalLatticePoint
import hyperspy.api as hs
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from orix import io, plot, sampling
from orix.crystal_map import create_coordinate_arrays, CrystalMap, Phase, PhaseList
from orix.quaternion import Rotation
from orix.vector import Vector3d
from scipy.spatial import distance as ssd

import kikuchipy as kp

from pyebsdindex import ebsd_index, pcopt

# Directories
dir_mp = "/home/hakon/kode/emsoft/emdata/crystal_data"
dir_data = "/home/hakon/phd/data/p/prover/300c/1"
dir_nordif = os.path.join(dir_data, "nordif")
dir_kp = os.path.join(dir_data, "kp")

# Matplotlib
plt.rcParams["font.size"] = 12
savefig_kw = dict(bbox_inches="tight", pad_inches=0, dpi=150)

# Print today
print(date.today())

2022-01-20


# Pre-correction

Load data

In [2]:
s = kp.load(os.path.join(dir_kp, "patterns_dewrap.h5"), lazy=True)
sig_shape = s.axes_manager.signal_shape[::-1]
print(s.axes_manager)

<Axes manager, axes: (919, 919|96, 96)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    919 |      0 |       0 |     0.1 |     um 
               y |    919 |      0 |       0 |     0.1 |     um 
---------------- | ------ | ------ | ------- | ------- | ------ 
              dx |     96 |        |       0 |       1 |     um 
              dy |     96 |        |       0 |       1 |     um 


Mean intensity map

In [10]:
s_mean = s.mean(axis=s.axes_manager.signal_axes)
s_mean.compute()

[########################################] | 100% Completed |  9.5s


In [11]:
map_mean1 = s_mean.data
plt.imsave(os.path.join(dir_kp, "maps_mean.png"), map_mean1, cmap="gray")
map_mean2 = kp.pattern.rescale_intensity(
    map_mean1, in_range=np.percentile(map_mean1, q=(1, 99))
)
plt.imsave(os.path.join(dir_kp, "maps_mean_q1_q99.png"), map_mean2, cmap="gray")

Virtual backscatter electron images

In [12]:
vbse_gen = kp.generators.VirtualBSEGenerator(s)

RGB image

In [15]:
vbse_gen.grid_shape = (5, 5)
red = (2, 1)
green = (2, 2)
blue = (2, 3)
vbse_grid_plot = vbse_gen.plot_grid(
    rgb_channels=[red, green, blue], pattern_idx=(0, 0)
)
vbse_grid_plot._plot.signal_plot.figure.savefig(
    os.path.join(dir_kp, "vbse5x5_grid_plot.png"), **savefig_kw
)

In [17]:
vbse_rgb = vbse_gen.get_rgb_image(r=red, g=green, b=blue)
vbse_rgb.save(os.path.join(dir_kp, "vbse5x5_rgb.png"))

2022-01-20 15:05:14,104 - hyperspy.io - WARNING - `signal_type='VirtualBSEImage'` not understood. See `hs.print_known_signal_types()` for a list of installed signal types or https://github.com/hyperspy/hyperspy-extensions-list for the list of all hyperspy extensions providing signals.
2022-01-20 15:05:18,832 - hyperspy.io - WARNING - `signal_type='VirtualBSEImage'` not understood. See `hs.print_known_signal_types()` for a list of installed signal types or https://github.com/hyperspy/hyperspy-extensions-list for the list of all hyperspy extensions providing signals.
2022-01-20 15:05:23,434 - hyperspy.io - WARNING - `signal_type='VirtualBSEImage'` not understood. See `hs.print_known_signal_types()` for a list of installed signal types or https://github.com/hyperspy/hyperspy-extensions-list for the list of all hyperspy extensions providing signals.


Grid

In [18]:
vbse_grid = vbse_gen.get_images_from_grid()

2022-01-20 15:05:29,797 - hyperspy.io - WARNING - `signal_type='VirtualBSEImage'` not understood. See `hs.print_known_signal_types()` for a list of installed signal types or https://github.com/hyperspy/hyperspy-extensions-list for the list of all hyperspy extensions providing signals.
2022-01-20 15:05:34,611 - hyperspy.io - WARNING - `signal_type='VirtualBSEImage'` not understood. See `hs.print_known_signal_types()` for a list of installed signal types or https://github.com/hyperspy/hyperspy-extensions-list for the list of all hyperspy extensions providing signals.
2022-01-20 15:05:40,178 - hyperspy.io - WARNING - `signal_type='VirtualBSEImage'` not understood. See `hs.print_known_signal_types()` for a list of installed signal types or https://github.com/hyperspy/hyperspy-extensions-list for the list of all hyperspy extensions providing signals.
2022-01-20 15:05:44,721 - hyperspy.io - WARNING - `signal_type='VirtualBSEImage'` not understood. See `hs.print_known_signal_types()` for a li

In [19]:
for idx in np.ndindex(vbse_grid.axes_manager.navigation_shape):
    plt.imsave(
        os.path.join(dir_kp, f"vbse5x5_y{idx[1]}x{idx[0]}.png"),
        arr=vbse_grid.inav[idx].data,
        cmap="gray"
    )

Background correction

In [20]:
s.remove_static_background()

In [21]:
s.remove_dynamic_background()

In [22]:
w = kp.filters.Window(window="gaussian", shape=(3, 3), std=1)
s.average_neighbour_patterns(window=w)

In [23]:
t0 = time()
s.save(os.path.join(dir_kp, "pattern_sda.h5"))
print((time() - t0) / 60, "min")

11.295029282569885 min


## Pre-indexing

In [24]:
s = kp.load(os.path.join(dir_kp, "pattern_sda.h5"), lazy=True)
print(s.axes_manager)

<Axes manager, axes: (919, 919|96, 96)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    919 |      0 |       0 |     0.1 |     um 
               y |    919 |      0 |       0 |     0.1 |     um 
---------------- | ------ | ------ | ------- | ------- | ------ 
              dx |     96 |        |       0 |       1 |     um 
              dy |     96 |        |       0 |       1 |     um 


In [37]:
iq = s.get_image_quality()
adp = s.get_average_neighbour_dot_product_map()

In [26]:
plt.imsave(os.path.join(dir_kp, "maps_iq.png"), arr=iq, cmap="gray")
plt.imsave(os.path.join(dir_kp, "maps_adp.png"), arr=adp, cmap="gray")

## Projection center from PyEBSDIndex

Load acquisition patterns

In [48]:
s = kp.load(os.path.join(dir_kp, "pattern_sda.h5"), lazy=True)
sig_shape = s.axes_manager.signal_shape[::-1]

In [45]:
nav_shape = s.axes_manager.navigation_shape[::-1]
pattern_grid = (7, 7)
steps = np.floor(np.array(nav_shape) / np.array(pattern_grid)).astype(int)
start = np.floor(steps / 2).astype(int)
idx_r = np.arange(1, pattern_grid[0] + 1) * steps[0] - start[0]
idx_c = np.arange(1, pattern_grid[1] + 1) * steps[1] - start[1]
pattern_idx = np.array(np.meshgrid(idx_r, idx_c)).reshape((2, -1)).T

s2_data_shape = (np.prod(pattern_grid),) + sig_shape
s2_data = np.zeros(s2_data_shape, dtype=s.data.dtype)
for i, (r, c) in enumerate(pattern_idx):
    s2_data[i] = s.data[r, c]

s2 = kp.signals.EBSD(s2_data)
s2.axes_manager[0].name = "x"
nav_size = s2.axes_manager.navigation_size

In [46]:
s2.plot(navigator=None)

Load calibration patterns

In [49]:
s_cal0 = kp.load(os.path.join(dir_nordif, "Setting.txt"))
sig_shape_cal = s_cal0.axes_manager.signal_shape[::-1]

In [51]:
s_cal0.remove_static_background()
s_cal0.remove_dynamic_background()

Removing the static background:
[########################################] | 100% Completed |  0.1s
Removing the dynamic background:
[########################################] | 100% Completed |  0.3s


In [52]:
s_cal0.plot(navigator=None)

In [53]:
cal_al_idx = [10, 11, 12, 13, 14]
s_cal = kp.signals.EBSD(s_cal0.data[cal_al_idx])
s_cal.set_experimental_parameters(
    static_background=s_cal0.metadata.Acquisition_instrument.SEM.Detector.EBSD.static_background
)
s_cal.axes_manager[0].name = "x"
nav_size = s_cal.axes_manager.navigation_size

/home/hakon/kode/kikuchipy/kikuchipy/signals/ebsd.py:166: VisibleDeprecationWarning: Function `set_experimental_parameters()` is deprecated and will be removed in version 0.6.
  def set_experimental_parameters(


Obtain PC and orientations from PyEBSDIndex

In [55]:
indexer = ebsd_index.EBSDIndexer(
    phaselist=["FCC"],
    vendor="EDAX",
    PC=None,
    sampleTilt=70,
    camElev=0,
    patDim=sig_shape_cal[::-1]
#    patDim=sig_shape[::-1]
)

Load master pattern

In [56]:
energy = 17
mp = kp.load(
    os.path.join(dir_mp, "al", "al_mc_mp_20kv.h5"),
    projection="lambert",
    energy=energy,
    hemisphere="north",
)
mp.phase.name = "al"
al_phase = mp.phase

Find PC from single pattern

In [70]:
pc0 = (0.48, 0.82, 0.54)
pc = pcopt.optimize(s_cal.inav[0].data, indexer, PC0=pc0)
print(pc)

[0.47372871 0.81526954 0.54449875]


Find PC from all patterns

In [71]:
pc0 = (0.48, 0.82, 0.54)
pcs = np.zeros((nav_size, 3))
for i in range(nav_size):
    pcs[i] = pcopt.optimize(s_cal.inav[i].data, indexer, PC0=pc0)
pc = pcs.mean(axis=0)
print(pc)

[0.48806974 0.81207367 0.54568834]


In [72]:
data = indexer.index_pats(
    patsin=s_cal.data,
#    patsin=s2.data,
    patstart=0,
    npats=-1,
    clparams=None,
    PC=pc,
    verbose=1
)
rot = Rotation(data[0]["quat"][0])

Radon Time: 0.25747991100070067
Convolution Time: 0.016753684998548124
Peak ID Time: 0.003076690001762472
Band Label Time: 0.013839708000887185
Total Band Find Time: 0.2911553129997628
Band Vote Time:  1.043534128999454


In [73]:
detector = kp.detectors.EBSDDetector(
    shape=sig_shape_cal,
#    shape=sig_shape,
    pc=pc,
    sample_tilt=70,
    convention="tsl",
)
detector

EBSDDetector (240, 240), px_size 1 um, binning 1, tilt 0, azimuthal 0, pc (0.488, 0.188, 0.546)

Inspect geometrical simulations

In [74]:
rlp = ReciprocalLatticePoint(phase=al_phase, hkl=((1, 1, 1), (2, 0, 0), (2, 2, 0), (3, 1, 1)))
rlp2 = rlp.symmetrise()
simgen = kp.generators.EBSDSimulationGenerator(
    detector=detector,
    phase=al_phase,
    rotations=rot,
)
geosim = simgen.geometrical_simulation(reciprocal_lattice_point=rlp2)

markers = geosim.as_markers(pc=False, bands=True, zone_axes_labels=False, zone_axes=False)

#del s_cal.metadata.Markers
s_cal.add_marker(marker=markers, plot_marker=False, permanent=True)
s_cal.plot(navigator=None)

#del s2.metadata.Markers
#s2.add_marker(marker=markers, plot_marker=False, permanent=True)
#s2.plot(navigator=None)

Refine results from PyEBSDIndex

In [77]:
xmap = CrystalMap(rotations=rot, phase_list=PhaseList(al_phase))

In [78]:
# Calibration patterns
xmap_refined = s_cal.refine_orientation(xmap=xmap, detector=detector, master_pattern=mp, energy=energy)
_, detector_ref = s_cal.refine_projection_center(xmap=xmap_refined, detector=detector, master_pattern=mp, energy=energy)

# Acquisition patterns from grid
#xmap_refined = s2.refine_orientation(xmap=xmap, detector=detector, master_pattern=mp, energy=energy)
#_, detector_ref = s2.refine_projection_center(xmap=xmap_refined, detector=detector, master_pattern=mp, energy=energy)

Refinement information:
	Local optimization method: Nelder-Mead (minimize)
	Keyword arguments passed to method: {'method': 'Nelder-Mead'}
Refining 5 orientation(s):
[########################################] | 100% Completed |  3.0s
Refinement speed: 1 patterns/s
Refinement information:
	Local optimization method: Nelder-Mead (minimize)
	Keyword arguments passed to method: {'method': 'Nelder-Mead'}
Refining 5 projection center(s):
[########################################] | 100% Completed |  1.8s
Refinement speed: 2 patterns/s


Check geometrical simulations

In [79]:
simgen = kp.generators.EBSDSimulationGenerator(
    detector=detector_ref,
    phase=al_phase,
    rotations=xmap_refined.rotations,
)
geosim = simgen.geometrical_simulation(reciprocal_lattice_point=rlp2)
markers = geosim.as_markers(pc=False, bands=True, zone_axes_labels=False, zone_axes=False)

del s_cal.metadata.Markers
s_cal.add_marker(marker=markers, plot_marker=False, permanent=True)

#del s2.metadata.Markers
#s2.add_marker(marker=markers, plot_marker=False, permanent=True)

Check simulated patterns

In [80]:
sim_list = []
for i in range(detector_ref.pc.shape[0]):
    detector_i = detector_ref.deepcopy()
    detector_i.pc = detector_i.pc[0]
    sim_i = mp.get_patterns(
        rotations=xmap_refined.rotations[i],
        detector=detector_i,
        energy=energy,
        compute=True,
    )
    sim_list.append(sim_i)
sim = hs.stack(sim_list, axis=0)

Creating a dictionary of (1,) simulated patterns:
[########################################] | 100% Completed |  0.1s
Creating a dictionary of (1,) simulated patterns:
[########################################] | 100% Completed |  0.1s
Creating a dictionary of (1,) simulated patterns:
[########################################] | 100% Completed |  0.1s
Creating a dictionary of (1,) simulated patterns:
[########################################] | 100% Completed |  0.1s
Creating a dictionary of (1,) simulated patterns:
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s


In [81]:
ncc = kp.indexing.similarity_metrics.NormalizedCrossCorrelationMetric(
    n_experimental_patterns=nav_size,
    n_dictionary_patterns=nav_size,
)
#scores = ncc(s2.data, sim.data).compute()
scores = ncc(s_cal.data, sim.data).compute()
scores = np.diagonal(scores)

In [82]:
del s_cal.metadata.Markers
hs.plot.plot_signals((s_cal, sim), navigator=hs.signals.Signal1D(scores))

#del s2.metadata.Markers
#hs.plot.plot_signals((s2, sim), navigator=hs.signals.Signal1D(scores))

## Dictionary indexing

Done on Windows machine with above projection center.

## Refinement

Done on Windows machine with above projection center.